Студент Жилкин Федор, 344
# Домашняя работа 2: "Итерационные методы решения СЛАУ."
**Задания:**

1. Метод простой итерации (с уточнением по Люстернику)
2. Метод Зейделя (с уточнением по Люстернику)
3. Метод верхней релаксации
 
$ A = 
\left(\begin{array}{ccc|c} 
6.687233 & 0.80267  & -2.06459 & 0\\ 
0.80267 & 5.07816 & 0.48037  & 1\\
-2.06459 & 0.48037 & 4.02934 & 0
\end{array}\right)
$

In [1]:
import numpy as np

In [2]:
A = np.array([[6.687233, 0.80267, -2.06459], [0.80267, 5.07816, 0.48037], [-2.06459, 0.48037, 4.02934]])

In [3]:
B = np.array([[0.], [1.], [0.]])

In [4]:
# Необходимая точность
eps = 0.0001

In [13]:
x_real = list(map(lambda x: x[0], np.linalg.solve(A, B)))
x_real

[-0.038609580502971634, 0.20723292452262765, -0.04448903139560444]

## Метод простой итерации

In [23]:
def iterative_method(A, B):
    
    n = len(A)
    beta = np.zeros(n)
    alpha = np.zeros((n, n))
    x = np.zeros(n)
    
    # Заполняем матрицы alpha и beta
    for i in range(0, n):
        beta[i] = B[i][0] / A[i][i]
        for j in range(0, n):
            if (i == j):
                alpha[i][j] = 0
            else:
                alpha[i][j] = - A[i][j]/A[i][i]

    # Нулевое приближение
    x = np.copy(beta)  
    
    # Количество итераций
    i = 0 
    
    # Итерационное приближение
    while(False in list(map(lambda x: x < eps, abs(x - x_real)))):
        tmp = np.copy(x)
        x = beta + np.dot(alpha, tmp)
        i += 1
    return x, i

In [24]:
x1, i1 = iterative_method(A, B)
x1, i1

(array([-0.03853631,  0.20718989, -0.04439994]), 8)

**Ответ:**
$ X = 
\begin{pmatrix}
 -0.03853631 \\
 0.20718989 \\
 -0.04439994
\end{pmatrix}
$

**Решение получено за 8 итераций**

## Метод Зейделя

In [25]:
def seidel(A, B):
    
    n = len(A)
    x = np.zeros(n)
    
    # Количество итераций
    i = 0 
    
    while(False in list(map(lambda x: x < eps, abs(x-x_real)))):
        x_new = np.copy(x)
        for i in range(n):
            s1 = sum(A[i][j] * x_new[j] for j in range(i))
            s2 = sum(A[i][j] * x[j] for j in range(i + 1, n))
            x_new[i] = (B[i] - s1 - s2) / A[i][i]

        x = x_new
        i += 1
    return x, i

In [26]:
seidel(A, B)

(array([-0.03852571,  0.20719954, -0.04444207]), 3)

**Ответ:**
$ X = 
\begin{pmatrix}
 -0.03852571 \\
 0.20719954 \\
 -0.04444207
\end{pmatrix}
$

**Решение получено за 3 итерации**

## Метод верхней релаксации

In [29]:
def SOR(A, B):
    
    n = len(A)
    x = np.zeros(n)
    
    # Количество итераций
    i = 0 
    
    # Параметр релаксации
    w = 1.25
    
    while(False in list(map(lambda x: x < eps, abs(x-x_real)))):
        x_new = np.copy(x)
        for i in range(n):
            s1 = sum(A[i][j] * x_new[j] for j in range(i))
            s2 = sum(A[i][j] * x[j] for j in range(i + 1, n))
            
            # Модификация шага метода Зейделя
            x_new[i] = w*(B[i] - s1 - s2) / A[i][i] + (1 - w)*x[i]

        x = x_new
        i += 1
    return x, i

In [30]:
SOR(A,B)

(array([-0.03864406,  0.20716288, -0.04450536]), 3)

**Ответ:**
$ X = 
\begin{pmatrix}
 -0.03864406 \\
 0.20716288 \\
 -0.04450536
\end{pmatrix}
$

**Решение получено за 3 итерации**